# Reproducing and trying to solve the bug in [this ticket](https://github.com/orgs/OpenMined/projects/81/views/1?pane=issue&itemId=28179093) - second notebook

In [1]:
import syft as sy
sy.requires(">=0.8.1-beta")
import numpy as np

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0


In [2]:
!ls /tmp/*.sqlite

/tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite


If `node` is not launched with `reset=True`, we will get the error in the ticket. Not using `reset=True`, we will use the existing `sqlite` database (`/tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite`)

In [3]:
node = sy.orchestra.launch(name="test-domain-1", 
                           port=8080, dev_mode=True)
domain_client = node.login(email="info@openmined.org", 
                           password="changethis")
guest_client = node.client.guest()
guest_client

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [202718]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8080): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: HTTPConnection: http://localhost:8080>

In [4]:
assert domain_client.credentials != guest_client.credentials

#### Make a plan

In [5]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    z = y.prod()
    return z

In [6]:
plan_ptr = my_plan.send(guest_client)

In [7]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [8]:
input_ptr = domain_client.api.services.action.set(input_obj)

In [9]:
pointer = guest_client.api.services.action.get_pointer(input_ptr.id)
pointer

```python
TwinPointer(Mock)
```
array([[1, 1, 1],
       [1, 1, 1]])

In [10]:
res_ptr = plan_ptr(x=pointer)

In [11]:
res_ptr

```python
Pointer
```
64

In [12]:
res_ptr.request(guest_client)

```python
class Request:
  id: str = 6a3904ef94df42c0b1d4e2d4bb482530
  requesting_user_verify_key: str = e48d4d8ec6bdaded3a73bec1f0c0e2cc8d87b68379ac27b39948ff9c69577844
  approving_user_verify_key: str = None
  request_time: str = 2023-05-16 20:49:04
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "f133bada7e33a1390671d1e8d82f0a5ee3ff11ee6b8304cf971936d4333209ee"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

In [13]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,4055dd52c50a4dba9aa7a7e38b6a3d87,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:49:04,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,5bc7391582da4750b1f859a3ca6bf108,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:47:12,<<class 'syft.service.request.request.Request'...
2,syft.service.message.messages.Message,2377d84165664e86a20ff238668c60f0,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:47:13,<<class 'syft.service.request.request.Request'...


In [14]:
domain_client.api.services.messages[0].linked_obj

```python
class LinkedObject:
  id: str = cbdfe506767a45cb87d0550c10fd225b
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  service_type: str = <class 'syft.service.request.request_service.RequestService'>
  object_type: str = <class 'syft.service.request.request.Request'>
  object_uid: str = 6a3904ef94df42c0b1d4e2d4bb482530

```

In [18]:
domain_client.api.services.request

Exception: syft.user.RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69 not in TYPE_BANK